In [14]:
import pandas as pd

In [15]:
df = pd.read_csv("cumulative.csv")
df = df.drop(columns=['rowid', 'kepid', 'kepoi_name', 'koi_pdisposition', 'koi_score', 'koi_tce_delivname', 'kepler_name'])
# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')
# Drop the null rows
df = df.dropna()
df.head()

,koi_disposition,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
0,CONFIRMED,0,0,0,0,9.488036,2.775000e-05,-2.775000e-05,170.538750,0.002160,...,-81.0,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
1,CONFIRMED,0,0,0,0,54.418383,2.479000e-04,-2.479000e-04,162.513840,0.003520,...,-81.0,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
2,FALSE POSITIVE,0,1,0,0,19.899140,1.494000e-05,-1.494000e-05,175.850252,0.000581,...,-176.0,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436
3,FALSE POSITIVE,0,1,0,0,1.736952,2.630000e-07,-2.630000e-07,170.307565,0.000115,...,-174.0,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597
4,CONFIRMED,0,0,0,0,2.525592,3.761000e-06,-3.761000e-06,171.595550,0.001130,...,-211.0,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509


In [16]:
from sklearn.model_selection import train_test_split

y = df["koi_disposition"]
X = df.drop(columns=['koi_disposition'])
X_train, X_test, y_train, y_test = train_test_split(X,y, random_state = 1, stratify=y)

In [17]:
X_train.head()

,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,koi_time0bk_err2,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
5964,1,0,0,0,252.044440,0.027490,-0.027490,265.2010,0.0494,-0.0494,...,-136.0,4.621,0.041,-0.035,0.664,0.057,-0.059,292.79022,41.948639,15.884
9410,1,0,0,0,371.518520,0.015790,-0.015790,317.6836,0.0339,-0.0339,...,-206.0,4.377,0.101,-0.203,1.089,0.364,-0.145,293.06400,45.034210,13.731
4204,0,0,1,0,8.038670,0.000114,-0.000114,135.3098,0.0123,-0.0123,...,-181.0,4.485,0.050,-0.200,0.975,0.282,-0.101,290.51785,41.238762,14.999
5933,0,0,0,0,18.782160,0.000406,-0.000406,147.8508,0.0148,-0.0148,...,-167.0,4.488,0.048,-0.290,0.940,0.386,-0.087,291.76413,41.860130,14.043
6996,1,0,1,0,12.429716,0.000472,-0.000472,141.2846,0.0420,-0.0420,...,-200.0,4.534,0.037,-0.213,0.905,0.281,-0.088,297.52072,40.585419,15.842


In [18]:
from sklearn.preprocessing import MinMaxScaler
X_scaler = MinMaxScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [19]:
from sklearn.svm import SVC
model1 = SVC(kernel='linear')
model1.fit(X_train_scaled, y_train)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [30]:
print("Test Acc: %3f" % model1.score(X_test_scaled, y_test))

Test Acc: 0.846295


In [20]:
print(f"Training Data Score: {model1.score(X_train_scaled, y_train)}")
print(f"Testing Data Score: {model1.score(X_test_scaled, y_test)}")

Training Data Score: 0.8479719426654467
Testing Data Score: 0.8462946020128088


In [21]:
from sklearn.model_selection import GridSearchCV
param_grid = {'C': [1,5,10], 'gamma':[0.0001, 0.001, 0.01]}
grid1 = GridSearchCV(model1, param_grid)

In [49]:
grid1.fit(X_train_scaled, y_train)


GridSearchCV(cv=None, error_score=nan,
             estimator=SVC(C=1.0, break_ties=False, cache_size=200,
                           class_weight=None, coef0=0.0,
                           decision_function_shape='ovr', degree=3,
                           gamma='scale', kernel='linear', max_iter=-1,
                           probability=False, random_state=None, shrinking=True,
                           tol=0.001, verbose=False),
             iid='deprecated', n_jobs=None,
             param_grid={'C': [1, 5, 10], 'gamma': [0.0001, 0.001, 0.01]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

In [35]:
print("Test Acc: %3f" % grid1.score(X_test_scaled, y_test))

Test Acc: 0.875572


In [50]:
print(grid1.best_params_)
print(grid1.best_score_)

{'C': 10, 'gamma': 0.0001}
0.8708443796394485


In [51]:
import joblib
filename = 'svc.sav'
joblib.dump(model1, filename)

['svc.sav']

In [56]:
predictions = grid1.predict(X_test_scaled)
# predictions[:10]
print(y_test[:10])

7217    FALSE POSITIVE
873          CANDIDATE
189          CONFIRMED
6744         CANDIDATE
9220         CANDIDATE
2554    FALSE POSITIVE
8706    FALSE POSITIVE
2734    FALSE POSITIVE
5685    FALSE POSITIVE
8878    FALSE POSITIVE
Name: koi_disposition, dtype: object


In [60]:
from sklearn.metrics import classification_report
print(classification_report(y_test, predictions,
                            target_names=["candidate", "confirmed", 'false positive']))

precision    recall  f1-score   support

     candidate       0.83      0.64      0.73       528
     confirmed       0.73      0.86      0.79       568
false positive       0.98      1.00      0.99      1090

      accuracy                           0.88      2186
     macro avg       0.85      0.83      0.83      2186
  weighted avg       0.88      0.88      0.87      2186

